In [ ]:
#coding=utf-8
import cv2
import torch
import numpy as np
from torch import nn
from PIL import Image
import torchvision as tv 
from matplotlib import pyplot as plt

from utils import api

In [ ]:
gain_api = api.GAINAPI('saved_models/best_GAIN with_transform: True.pt')

In [ ]:
img = Image.open('data/IJCAI_2019_AAAC_train/00003/109afd81452d4c1d431be3eb2167b783.jpg').resize((224, 224))
img_np = np.array(img)
cam = gain_api(img)
mask = cam > 0.5

In [ ]:
plt.imshow(img_np * (1 - mask.reshape(224, 224, 1)))